In [ ]:
# !git clone https://github.com/dsa-playground/tvt2024.git
# %cd /content/tvt2024/
# !git pull
# !pip install upgrade -r requirements.txt -t "tvt2024"
# !pip install pyaml-env

In [ ]:
# Imports
import datetime
import warnings
import pandas as pd
from scripts.main import laad_data, bekijk_ziekteverzuim, voorspel_met_voortschrijdend_gemiddelde, voorspel_met_regressie, pas_voortschrijdend_gemiddelde_toe, pas_regressie_toe, onderzoek_afwijkingen, bereken_metrieken, voorspel, opties_berekenen, optie_1, optie_2, optie_3

## Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)
datum_vandaag = datetime.datetime.now().strftime("%Y-%m-%d")
# Disable warnings
warnings.filterwarnings('ignore')

# 1. Data en doel

Jullie zijn allen werkzaam bij zorgorganisatie 'Zorgello' in de afdeling Analytics. De bestuurder komt binnen stormen:

*Morgen heb ik een vergadering met de Raad van Toezicht. Vorig jaar hebben ze me flink aan de tand gevoeld over het ziekteverzuim. Ik heb voor de vergadering morgen een forecast nodig, zodat ik kan laten zien dat we vroegtijdig acteren. Kunnen jullie deze even snel maken? Het gaat om het voorspellen van het ziekteverzuimpercentage van ons personeel.* 

**Opgave 1.1.** De bestuurder geeft direct het doel aan: Voorspel het verzuimpercentage. Hiermee ga je voorbij aan de eerste stap: Wat kun je bedenken bij de data die geregistreerd wordt om te komen tot een ziekteverzuimpercentage?

<!-- 
Ziekteverzuimpercentage is een berekening van de som van het verzuim per dag gedeeld door het aantal medewerkers in dienst per dag. 
Oftewel:
•	Aantal medewerkers: In dienst (ja/nee) per dag
•	Aantal ziek: Som verzuim per dag. Alle meldingen tussen ziektemelding en beter melding, soms zelfs met 'gedeeltelijk ziek', soms zelfs exclusief zwangerschapsverlof, e.d.
-->

Bekijk de data van het ziekteverzuimpercentage.

In [ ]:
data = laad_data()
bekijk_ziekteverzuim(data)

Het ziekteverzuimpercentage wisselt tussen de 7% en 10%. Het effect van uitgevallen medewerkers is een hogere werkdruk voor ingeroosterd personeel en wellicht de extra inzet van personeel niet in loondienst (duurder). 

#### Begrippen en definities
Wanneer dezelfde variabele op verschillende momenten gemeten worden ontstaat een tijdreeks. De tijdreeksen van ziekteverzuim heeft een dagelijkse meting (interval = dag). Een tijdreeks bestaat uit de volgende elementen:
* Trend: richting (stijgend/dalend)
* Seizoenspatroon/-patronen: terugkerend patroon (bepaalde momenten standaard hoger/lager)
* Ruis: afwijkingen niet te verklaren door trend of patroon

**Opgave 1.2.** Om de elementen van de tijdreeks beter uit te leggen, willen we dat je even nadenkt over temperatuur. De metingen van temperatuur kunnen beschouwd worden als een tijdreeks. Laten we kijken hoe goed de algemene kennis van temperatuur is en de begrippen en definities van tijdreeksen geland zijn. 

a. Wat weten we van de trend in temperatuur?

<!-- Temperatuur neemt toe door invloed mens: Stijgende trend. -->

b. Welke seizoen patronen kun je herkennen in de temperatuur?

<!-- - Dag patroon ('s nachts kouder dan overdag, invloed zonopwarming)
- Jaarpatroon (4 seizoenen, draaiing aarde rond de zon) -->

c. Wat zijn mogelijke factoren die ruis veroorzaken?

<!-- Bewolking, wind- en waterstromingen, zonnevlammen, smog/uitstoot, ... -->

# 2. Kaders van model

Een voorspelling is een schatting voor de toekomst. Afhankelijk van welke eisen je stelt, kun je een model kiezen. Denk aan:
* Welke data kan het model gebruiken? Is deze data er historisch én in de toekomst?
* Hoe ver in de toekomst moet het model voorspellen?
* Op welk detailniveau heb ik een voorspelling nodig?

We kiezen ervoor in deze workshop om de rest van het huidig jaar (2024) te voorspellen. Daarnaast houden we de dataset simpel, we gaan enkel uit van de historische reeks van het ziekteverzuimpercentage en de bijbehorende datum.

Om in deze workshop niet te verzanden in complexe modellen, hebben we gekozen voor twee ('eenvoudige') modellen:
* Het voortschrijdend gemiddelde
* Het regressiemodel


#### *Het voortschrijdend gemiddelde*
Het voortschrijdend gemiddelde maakt gebruik van de historische waarden van het onderwerp. Op basis van een aantal voorgaande waarden wordt een gemiddelde berekend. Dit gemiddelde is de voorspeller voor de toekomst. 

**Opgave 2.1.** Stel het voortschrijdend gemiddelde wordt bepaald op basis van de 4 voorgaande waarden. Hoe zien de eerste 2 getallen (a en b) eruit na deze reeks:

*9, 16, 12, 8, 12, a, b*

a = ...

b = ...


<!-- a = (16 + 12 + 8 + 12) / 4 = 12

b = (12 + 8 + 12 + 12) / 4 = 11 -->


Laten we het voortschrijdend gemiddelde toepassen voor het ziekteverzuimpercentage. 


In [ ]:
voortschrijdend_gemiddelde = voorspel_met_voortschrijdend_gemiddelde(data=data)

**Opgave 2.2.** Wat valt je op aan de uitkomsten van het voortschrijdend gemiddelde model?

<!-- - 1e week in de traindata heeft geen waarde.
- Na een periode van voorspellingen onstaat een constante waarde.  -->


#### *Regressie*
Er zijn ook algoritmes welke niet enkel kijken naar de historische waarden van het onderwerp dat je probeert te voorspellen. Deze algoritmes maken gebruik van verklarende variabele(n). Het eenvoudigste model is een regressiemodel. Wat dit model doet een regressielijn bepalen: best passende lijn om de relatie tussen verklarende variabelen (x) en te voorspellen variabelen (y) te bepalen. 

Laten we een eenvoudige regressie met 1 variabele (volgordelijkheid datums) toepassen:

In [ ]:
regressie = voorspel_met_regressie(data=data)

**Opgave 2.3.** Welke verklarende variabele heeft deze tijdreeks?

<!-- Datumreeks / dag -> deze wordt onder water numeriek gemaakt. -->

**Opgave 2.4.** Welke van deze modellen zou jij gebruiken en waarom?

# 3. Evaluatie metriek

Elk model geeft een voorspelling. De visuele check van de voorspelling naast de eigenlijke meting is hierboven getoond. Daarnaast kun je kijken naar de afwijking op zichzelf. 

#### *De afwijking (error)*
Laten we de reeks van errors voor elk model eens bekijken voor de testperiode.  

In [ ]:
onderzoek_afwijkingen(list_of_dfs=[voortschrijdend_gemiddelde, regressie])

Bij de toegepaste modellen wordt voor de train periode natuurlijk een 'optimum' gezocht binnen de grenzen van een model. Maar afhankelijk van het onderwerp en het model zul je verschillen zien.

**Opgave 3.1.** Waar kijk je naar bij de grafiek van de afwijking?

<!-- 
* Ligt de error rond een bepaalde vaste afwijking? 
* Neemt de grootte van de afwijking af/toe over tijd? Dit noemen we 'modeldrift'
* Zijn er extremen te herkennen? Als de error grote afwijkingen kent, dan is er in de oorspronkelijke data heel veel ruis. De vraag is of deze ruis te corrigeren is (in verleden en toekomst). Zo niet, dan moet je je afvragen welke afwijking hinderlijk zijn in het acteren op een voorspelling.  
-->

#### *Metrieken*
Om subjectiviteit eruit te halen is het belangrijk stil te staan bij het selecteren van een goede metriek. Een metriek is een getalsmatige uitdrukking van hoe goed/slecht je model presteert. Er zijn ontzettend veel metrieken, elk met een specifiek inzicht en doel. Het toepassen van meerdere metrieken voor de beoordeling van een model wordt ook aangeraden.

Voor deze workshop hebben we 3 metrieken gekozen welke inzicht geven in de prestatie van de modellen:
* **Juistheid**: Deze geeft aan hoe goed de voorspelling is voor de gehele testperiode. 1 is gelijk, kleiner dan 1 is te laag en groter dan 1 is te hoog. 
* **Maximale afwijking**: Voor elke dag is er een voorspelling en een realisatie in de testperiode. Deze metriek geeft de maximale afwijking weer. Hiermee zou je de maximale 'schade' kunnen bepalen als een voorspelling gebruikt wordt voor een vervolg(proces).
* **Gemiddelde absolute afwijking**: Deze wordt berekend met Mean Absolute Error. Deze metriek berekend het gemiddelde van de verschillen in absolute waarde (alles wordt positief getal). Het geeft daarmee inzicht in hoe groot de gemiddelde afwijking echt is. Hoe dichter bij 0, hoe beter het model presteert. 




In [ ]:
bereken_metrieken(list_of_dfs=[voortschrijdend_gemiddelde, regressie])

**Opgave 3.2.** Bij meerdere metrieken kan het voorkomen dat voor sommige metrieken model A beter presenteert en voor andere metrieken model B. Hoe bepaal je dan welk model je moet kiezen? 

<!-- Je kiest metrieken die aansluiten bij je behoefte. Daarnaast is het belangrijk om een volgorde toe te kennen welke metriek je het belangrijkste vindt en bij welke afwijking een metriek doorslaggevend is. Ook hierbij: Wat is de impact in een vervolgproces!  -->

# 4. Impact & gevolg

Na een eerste beoordeling volgt een actie. Afhankelijk van welke actie, dient er rekening gehouden worden met de impact en het gevolg op het model. 

De eerste actie welke we beschrijven is voor de forecaster. In de bovenstaande voorbeelden zijn standaard instellingen gekozen voor het model. De voorspelling die het oplevert voldoen nog niet. Door het aanpassen van de instellingen, kun je tot een beter model komen. 

**Instellingen voor het voortschrijdend gemiddelde**

Het voortschrijdend gemiddelde kent in deze toepassing twee variabelen:
* *vensterlengte*: Het aantal dagen voorafgaand waar het gemiddelde voor berekend wordt. Oftewel, bij 7 wordt het gemiddelde genomen van 7 voorafgaande dagen (lees: week). De minimumwaarde voor vensterlengte is 1. 
* *verschuiving*: Het aantal dagen dat teruggekeken moet worden. Oftewel, bij 28 (4-wekelijks patroon) gaat het model 4 weken terug en neemt dan de voorgaande window_size om het gemiddelde te bepalen. De minimumwaarde voor verschuiving is 0.

Zie tevens het plaatje hieronder:

![Instellingen bij voortschrijdend gemiddelde](https://raw.githubusercontent.com/dsa-playground/tvt2024/main/images/voortschrijdend_gem_instellingen.png)

**Instellingen voor regressie**

Het regressiemodel kent in deze toepassing drie variabelen:
* *jaarlijks_patroon*: Denk je dat er een jaarpatroon in het ziekteverzuim is? Dan kies je voor True. Zo niet, dan kies je False.
* *wekelijks_patroon*: Denk je dat er een weekpatroon in het ziekteverzuim is? Dan kies je voor True. Zo niet, dan kies je False.
* *graad*: De graad is een geheel getal, beginnend bij 1. Afhankelijk van welk getal je kiest, bepaald hoe gezocht wordt naar de best passende lijn.

Zie tevens plaatje hieronder:

![Instellingen bij regressie](https://raw.githubusercontent.com/dsa-playground/tvt2024/main/images/regressie_instellingen.png)

**Opgave 4.1.** In de onderstaande tabel staan verschillende instellingen. Door de cel hieronder te draaien worden de opties uitgerekend voor de testperiod (laatste jaar). Dit levert:
- Een grafiek van het ziekteverzuim en de voorspelling van de verschillende opties. Door te klikken op de lijnen in de legenda kun je ze (tijdelijk) weghalen. 
- Een tabel met metrieken (eronder) van de verschillende opties. 

Welke van de volgende instelling levert het beste resultaat op? 

|Optie 1 | Optie 2 | Optie 3 | Optie 4 |
|:---|:---|:---|:---|
|Voortschrijdend gemiddelde:<ul><li>vensterlengte = 1</li><li>verschuiving = 365</li></ul>|Voortschrijdend gemiddelde:<ul><li>vensterlengte = 7</li><li>verschuiving = 365</li></ul>|Regressiemode:<ul><li>jaarlijks_patroon=True</li><li>wekelijks_patroon=False</li><li>graad=1</li></ul>|Regressiemode:<ul><li>jaarlijks_patroon=True</li><li>wekelijks_patroon=True</li><li>graad=2</li></ul>|

<!-- Op basis van juistheid optie 3. Als afwijking als tweede metriek belangrijk is, dan zou optie 2 de beste optie zijn, aangezien deze zowel een goede score geeft voor juistheid als gemiddelde absolute afwijking.  -->



In [ ]:
opties_berekenen(data)

#### Voorspel

Als je een model en de (beste) instellingen gevonden hebt, dan rest uiteraard het genereren van de voorspelling. 
Door het draaien van de onderstaande cel wordt een voorspelling gemaakt tot 1 januari 2025. 

In [ ]:
voorspelling = voorspel(
    data=data,
    voorspellen_tot_datum='2025-01-01'
)

Nu er een voorspelling is die voldoet, kan men uiteraard de voorspelling gebruiken. In het geval van ziekteverzuimpercentage, kun je natuurlijk verschillende keuzes maken als bestuurder.

**Opgave 4.2.** Afhankelijk van welke actie je uitzet, moet je nadenken over de impact op het model. Het vermoeden ontstaat dat een groot deel van de ziekte verklaard wordt door werkdruk. Besloten wordt om elk team uit te breiden met 1 FTE. Wat is de impact op het model?

<!-- 

Scenario 0: Je voert geen proceswijziging door, geen impact op model. 

Scenario 1: Je voert een wijziging door waardoor het ziekteverzuim mogelijk veranderd. Bij complexere modellen heb je de mogelijkheid om een variabele toe te voegen die per dag aangeeft welk beleid je hebt gevoerd. Het model kan hiermee dan rekening houden.

Scenario 2: Hierbij ga je de berekening van je doelvariabele aanpassen. Als forecaster doe je er verstandig aan om een nieuwe tijdreeks te berekenen volgens deze nieuwe definitie, daar een model op te trainen en dat te gebruiken voor voorspellingen. 

-->

**Bonusvragen**

**Opgave 4.3.** Vraag voor gevorderde(n): Waarom is er geen maandelijks_patroon als instelling bij regressie?

<!-- Maanden zijn onderling niet te vergelijken. Het aantal dagen verschilt per maand en niet elke dag begint op dezelfde weekdag. Bij deze data is er geen onderliggend proces dat vraagt om een maandpatroon. -->

**Opgave 4.4.** Vraag voor de snellere forecasters: Wat zijn de beste instellingen die jij kan vinden?

*Hint*: Probeer eerst met grotere wijzigingen een beeld te krijgen waar je het beste antwoord kunt vinden. 

* Model = .....?

* Voortschrijdend gemiddelde
    - vensterlengte = .....?
    - verschuiving = .....?

* Instellingen regressiemodel
    - jaarlijks_patroon = .....?
    - wekelijks_patroon = .....?
    - graad = .....?

In [ ]:
## Instellingen model voortschrijdend gemiddelde
vensterlengte = 7
verschuiving = 0

## Toepassen instellingen in model
df_voortschrijdend_gemiddelde = pas_voortschrijdend_gemiddelde_toe(
    data=data,
    vensterlengte=vensterlengte,
    verschuiving=verschuiving,
    zie_traintest_periodes=True
)

## Berekenen metrieken
bereken_metrieken(list_of_dfs=[df_voortschrijdend_gemiddelde])

In [ ]:
## Instellingen regressiemodel
jaarlijks_patroon=False
wekelijks_patroon=False
graad = 1

## Toepassen instellingen in model
df_regressie = pas_regressie_toe(data=data,
    jaarlijks_seizoenspatroon=jaarlijks_patroon,
    wekelijks_seizoenspatroon=wekelijks_patroon,
    graad=graad,
    zie_traintest_periodes=True
)

## Berekenen metrieken
bereken_metrieken(list_of_dfs=[df_regressie])